In [20]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
nltk.download('punkt')
nltk.download('all')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora

False

In [0]:
# data_path='./'  
# os.chdir(data_path)
files=glob.glob('*.txt')

sents=[]
tagsP=[]
tagsF=[]
emptyin=[]
for f in files:
    file = open(f)
    df= pd.read_csv(file,delimiter='\t',encoding='utf-8')
    emptyin=df[df['Past'].isnull()].index.tolist()
    st=np.delete(np.array(df['sentence'].values),emptyin)
    ft=np.delete(np.array(df['Future'].values),emptyin)
    pt=np.delete(np.array(df['Past'].values),emptyin)
    sents.extend(list(st))
    tagsF.extend(list(ft))
    tagsP.extend(list(pt))
    print(emptyin)
X=np.asarray(sents)
yf=np.asarray(tagsF)
yp=np.asarray(tagsP)
print(len(X))
print(len(yf))
print(len(yp))

[103, 266, 280, 301, 314, 392, 412, 474, 664, 685, 688, 778, 875, 926, 975]
[49, 82, 195, 258, 533, 541, 577, 648, 726, 942]
[51, 90, 214, 273, 304, 404, 458, 511, 527, 640, 664, 769, 911, 995]
2961
2961
2961


In [0]:
f='projectivity.txt'
file=open(f)
df= pd.read_csv(file,delimiter='\t',encoding='utf-8')
emptyin=df[df['Past'].isnull()].index
dt=df.loc[:, 'id':'Future']
dt=dt.drop(emptyin)
print(len(dt))
# dt=np.delete(np.array(df.loc[:, 'A':'E'].values),emptyin)



2961


In [0]:
def split_data_and_store(df):
  train, validation, test = np.split(df, [int(.6*len(df)), int(.8*len(df))])
  train.to_csv('train', sep='\t')
  validation.to_csv('validation', sep='\t')
  test.to_csv('test', sep='\t')
  print('Done!')
  

In [0]:
split_data_and_store(dt)

In [0]:
stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'a', 'an', 'the', 'and', 'of', 'for', 'with', 'about', 'between', 'into', 'through', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'under', 'here', 'there', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'so', 'than', 'too', 'very', 'just']

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt


In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word for (word,pos) in nltk.pos_tag(tokens)]#+' '+pos
#     document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     print(document)
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs

In [0]:
# documents = []
# stemmer = WordNetLemmatizer()
# for sen in range(0, len(X)):
#   document = re.sub(r'\W', ' ', str(X[sen]))
#   document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
#   document = re.sub(r'^"', '', document)
#   document = re.sub(r'"$', '', document)
#   document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
#   document = re.sub(r'\s+', ' ', document, flags=re.I)
#   document = re.sub(r'^b\s+', '', document)
#   document = document.lower()
#   #     document = document.split()
#   #     document = [stemmer.lemmatize(word) for word in document]
#   #     document = ' '.join(document)
#   documents.append(document)
# print(len(documents))

986


In [15]:
# import numpy as np

with open(datapath+"glove.6B.300d.txt", "r", encoding='utf-8', errors='ignore') as lines:
    w2v = {line.split()[0]: np.array(list(map(float, line.split()[1:]))) for line in lines}
print(w2v['of'])
# def readWordvectors(wordvectorfile):
#   wordvectors = {}
#   vectorsize = 0
#   if ".gz" in wordvectorfile:
#     f = gzip.open(wordvectorfile, 'r')
#   else:
#     f = open(wordvectorfile, 'r',encoding='latin-1')
#   count = 0
#   for line in f:
#     if count == 0:
#       count += 1
#       continue
#     parts = line.split()
#     word = parts[0]
#     parts.pop(0)
#     wordvectors[word] = parts
#     vectorsize = len(parts)
#   f.close()
#   return [wordvectors, vectorsize]
# w2v,vectorsize= readWordvectors('vecs.txt')

[-7.6947e-02 -2.1211e-02  2.1271e-01 -7.2232e-01 -1.3988e-01 -1.2234e-01
 -1.7521e-01  1.2137e-01 -7.0866e-02 -1.5721e+00 -2.2464e-01  4.2690e-02
 -4.0180e-01  2.1006e-01  1.4288e-02  4.1628e-01  1.7165e-02  7.1732e-02
  6.9246e-03  1.8107e-01 -1.5412e-01  1.4933e-01 -3.0493e-02  2.9918e-01
  2.9479e-02 -3.6147e-02 -6.1125e-02  8.3918e-02 -1.2398e-01 -1.0077e-01
 -5.4142e-03  3.3710e-01 -2.5612e-01  4.4388e-01 -6.8922e-01  1.8020e-01
  3.4898e-01 -5.2284e-02 -2.6226e-01 -4.7109e-01  2.1647e-01 -4.0020e-01
 -4.9986e-02  1.1376e-02  5.4994e-01 -2.2791e-01  9.5873e-02  4.7693e-01
 -5.6727e-02 -1.7895e-01  1.1756e-01  1.4662e-01  4.8948e-02  1.3587e-01
 -9.3821e-02  4.5968e-01 -3.2062e-01  2.9911e-01  2.0656e-01 -1.8503e-01
 -2.7690e-01 -2.2545e-02  7.0698e-01 -2.3815e-01  1.6437e-01 -5.5044e-01
 -1.0615e-03  1.2266e-01  1.1898e-01  2.3985e-01  2.9815e-01  1.3207e-02
  1.6316e-01 -6.1334e-01 -3.7051e-01  1.9444e-01 -1.3621e-01 -3.0426e-01
 -3.7715e-01  6.5299e-02 -1.5995e-01 -5.6516e-01  7

In [16]:
len(w2v)

400000

In [0]:
import os    
from chardet import detect

def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read().decode('latin-1')
    return detect(rawdata)['encoding']

from_codec = get_encoding_type("vecs.txt")
print(from_codec)

In [0]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(list(w2v.values())[1])

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.sum([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [18]:
len(list(w2v.values())[1])

# list(w2v.keys())[:10]

300

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
datapath = 'gdrive/My Drive/data/'

In [27]:
#loading train and validation data
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
tr_st = sent_to_features(tr_st)
val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [28]:
print(tr_st[1])

friends and adversaries abroad were asking whether america had lost its nerve


In [0]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'a', 'an', 'the', 'and', 'of', 'for', 'with', 'about', 'between', 'into', 'through', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'under', 'here', 'there', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'so', 'than', 'too', 'very', 'just']


In [0]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer( stop_words=stop_words)  #, stop_words=stopwords.words('english')
documents = np.concatenate((tr_st,val_st))
print(len(documents))
vectorizer.fit(documents)


2368


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', '...t', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'so', 'than', 'too', 'very', 'just'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
tr_st = vectorizer.transform(tr_st)
val_st = vectorizer.transform(val_st)

In [0]:
print(tr_st.shape)
# print(X[0][:1000])

(1776, 39395)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer()#min_df=2, max_df=0.8, stop_words=stopwords.words('english')
documents = np.concatenate((tr_st,val_st))
print(len(documents))
tfidfconverter.fit(documents) 
tr_st = tfidfconverter.transform(tr_st)
val_st = tfidfconverter.transform(val_st)

2368


In [0]:
print(tr_st.shape)
# print(Xt[0][:1000])


(1776, 39471)


In [29]:
meanevconverter= MeanEmbeddingVectorizer(w2v)
tr_st = meanevconverter.transform(tr_st)
print('tranformed tr data: ', tr_st.shape)
val_st = meanevconverter.transform(val_st)
print('tranformed val data', val_st.shape)

tranformed tr data:  (1776, 300)
tranformed val data (592, 300)


In [0]:
print(tr_st[20])

In [0]:
# from sklearn.model_selection import train_test_split  
# Xtr, Xts, ytr, yts = train_test_split(X, yf, test_size=0.3, random_state=42)

In [0]:
from sklearn import svm

svc = svm.SVC(probability=False, kernel="linear", C=2.8, gamma=.0073,verbose=10)

In [31]:
svc.fit(tr_st,tr_ft)

[LibSVM]

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0073, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [0]:
yhat_ts = svc.predict(val_st)

In [0]:
yhat_tr = svc.predict(tr_st)

In [34]:
acc = np.mean(yhat_tr == tr_ft)
print('Training Accuaracy = {0:f}'.format(acc))

Training Accuaracy = 0.643581


In [35]:
acc = np.mean(yhat_ts == val_ft)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.636824


In [36]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
print(classification_report(val_ft, yhat_ts, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.62      0.42      0.50       256
     class 1       0.65      0.80      0.71       336

   micro avg       0.64      0.64      0.64       592
   macro avg       0.63      0.61      0.61       592
weighted avg       0.63      0.64      0.62       592



In [0]:
print("Future:")
unique, counts = np.unique(np.concatenate((tr_ft,val_ft)), return_counts=True)
print("Total: ",dict(zip(unique, counts)))
unique, counts = np.unique(tr_ft, return_counts=True)
print("Train: ",dict(zip(unique, counts)))
unique, counts = np.unique(val_ft, return_counts=True)
print("Test: ",dict(zip(unique, counts)))


Future:
Total:  {0.0: 1012, 1.0: 1356}
Train:  {0.0: 756, 1.0: 1020}
Test:  {0.0: 256, 1.0: 336}


In [0]:
# Xtr, Xts, ytrp, ytsp = train_test_split(X, yp, test_size=0.3, random_state=42)


In [0]:
svcp = svm.SVC(probability=True, kernel="linear", C=2.8, gamma=.0073,verbose=10)

In [38]:
svcp.fit(tr_st,tr_pt)

[LibSVM]

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0073, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [0]:
yhat_tsp = svcp.predict(val_st)

In [0]:
yhat_trp = svcp.predict(tr_st)

In [41]:
acc = np.mean(yhat_trp == tr_pt)
print('Training Accuaracy = {0:f}'.format(acc))

Training Accuaracy = 0.681306


In [42]:
acc = np.mean(yhat_tsp == val_pt)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.641892


In [0]:
target_names = ['class 0', 'class 1']
print(classification_report(val_pt, yhat_tsp, target_names=target_names))

In [0]:
print("Past:")
unique, counts = np.unique(np.concatenate((tr_pt,val_pt)), return_counts=True)
print("Total: ",dict(zip(unique, counts)))
unique, counts = np.unique(tr_pt, return_counts=True)
print("Train: ",dict(zip(unique, counts)))
unique, counts = np.unique(val_pt, return_counts=True)
print("Test: ",dict(zip(unique, counts)))


Past:
Total:  {0.0: 1452, 1.0: 916}
Train:  {0.0: 1100, 1.0: 676}
Test:  {0.0: 352, 1.0: 240}


In [43]:
testfile=datapath+'test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
ts_st = sent_to_features(ts_st)



593


In [0]:

ts_st = vectorizer.transform(ts_st)

In [0]:
ts_st = tfidfconverter.transform(ts_st)


In [0]:
tr_val_st = vectorizer.transform(documents)

In [0]:
tr_val_st = tfidfconverter.transform(documents)

In [44]:
documents = np.concatenate((tr_st,val_st))
print(len(documents))
tr_val_st = meanevconverter.transform(documents)
ts_st = meanevconverter.transform(ts_st)

2368


In [45]:

svc2 = svm.SVC(probability=True, kernel="linear", C=0.1, gamma=0.001, verbose=10)
print(tr_val_st.shape,tr_ft.shape,val_ft.shape)
svc2.fit(tr_val_st,np.concatenate((tr_ft,val_ft)))


(2368, 300) (1776,) (592,)
[LibSVM]

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [46]:
yhat_testing = svc2.predict(ts_st)
accu = np.mean(yhat_testing == ts_ft)
print('Test Accuracy: ',accu)

Test Accuracy:  0.5059021922428331


In [47]:
target_names = ['class 0', 'class 1']
print(classification_report(ts_ft, yhat_testing, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00       293
     class 1       0.51      1.00      0.67       300

   micro avg       0.51      0.51      0.51       593
   macro avg       0.25      0.50      0.34       593
weighted avg       0.26      0.51      0.34       593



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
svc2 = svm.SVC(probability=True, kernel="linear", C=1, gamma=0.001, verbose=10)
svc2.fit(tr_val_st,np.concatenate((tr_pt,val_pt)))

[LibSVM]

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [49]:
yhat_testing = svc2.predict(ts_st)
acc = np.mean(yhat_testing == ts_pt)
print('Test Accuracy: ',acc)
target_names = ['class 0', 'class 1']
# print(classification_report(ts_pt, yhat_testing, target_names=target_names))

Test Accuracy:  0.5463743676222597
